# Data Preparation Notebook

<div class="alert alert-block alert-warning">
The objective of this notebook is to create the dataset (<bold> ../data/data_preparation_output.csv </bold>), used subsequenially in the analysis & data visualizaion
</div>


1. Data standardisation
    1. text
        - lowercase
        - ltrim
2. Data cleaning
    1. encoding as missing values
    2. outlier detection
3. Feature creation
    1. conversion rate
    2. competition in the market
    3. miscellaneous
4. Data enhancement
    1. stock prices
    2. country value

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import os

import process_gini as pg
import plots

In [124]:
path = "../individualNotebooks/skylab_instagram_datathon_dataset.csv"  # ASY local path
# path = "../data/skylab_instagram_datathon_dataset.csv" # gen
df = pd.read_csv(path, sep=";")
# df = pd.read_csv(path.replace("/", os.sep), sep=";") # for Windows

# Process data

## Data standardisation

In [125]:
# READJUST STRINGS
string_vars = ["compset_group", "compset", "business_entity_doing_business_as_name",
"legal_entity_name", "domicile_country_name", "ultimate_parent_legal_entity_name",
"primary_exchange_name"]

for varname in df.columns:
    # 1. remove leading and trailing whitespaces and convert to lowercase
    df[varname] = df[varname].apply(lambda x: x.strip().lower() if isinstance(x, str) else x)
    # 2. replace all double whitespaces with single whitespaces
    df[varname] = df[varname].apply(lambda x: x.replace("  ", " ") if isinstance(x, str) else x)

In [126]:
# Converting the 'period_end_of_week' column to datetime
df['period_end_date'] = pd.to_datetime(df['period_end_date'])

In [127]:
# REPLACE DOMICILE_COUNTRY_NAME

# replace the china;hong kong with hong kong
df['domicile_country_name'] = df['domicile_country_name'].replace('china;hong kong', 'hong kong')

# remove the sign ";" in column domicile_country_name
df['domicile_country_name'] = df['domicile_country_name'].str.replace(';', '')

# if empty, fill with nan
df['domicile_country_name'] = df['domicile_country_name'].replace('', np.nan)

In [128]:
# REPLACE PRIMARY_EXCHANGE_NAME

# remove the sign ";" in column domicile_country_name
df['primary_exchange_name'] = df['primary_exchange_name'].str.replace(';', '')

# if empty, fill with nan
df['primary_exchange_name'] = df['primary_exchange_name'].replace('', np.nan)

In [129]:
# REPLACE ULTIMATE_PARENT_LEGAL_ENTITY_NAME
df['ultimate_parent_legal_entity_name'] = df['ultimate_parent_legal_entity_name'].replace('Anheuser-Busch;Anheuser-Busch', 'Anheuser-Busch')

### Filtering

In [130]:
# remove unnecessary columns
# Here: 'period', 'calculation_type'
df = df.drop(columns=['period', 'calculation_type'])

<div class="alert alert-block alert-danger">
<b>Assumption:</b> All Brands represent the aggregated on all brands in the corresponding compset => delete it </b>
</div>
(checked in the sell below)

In [107]:
#varnames_resp = ['followers','pictures', 'videos', 'comments']

#df1 = df.loc[df["business_entity_doing_business_as_name"] == "all brands"].sort_values(['period_end_date','compset_group', 'compset'])[['period_end_date','compset_group', 'compset', 'followers','pictures', 'videos', 'comments']]
#df2 = df.loc[df["business_entity_doing_business_as_name"] != "all brands"].sort_values(['period_end_date','compset_group', 'compset'])[['period_end_date','compset_group', 'compset', 'followers','pictures', 'videos', 'comments']]
#df2 = df2.groupby(['period_end_date','compset_group', 'compset']).sum()

#df1.join(df2, on=['period_end_date','compset_group', 'compset'], rsuffix='_compare')

In [131]:
# Remove "All_Brands" 
df = df[df["business_entity_doing_business_as_name"] != "all brands"]

<div class="alert alert-block alert-danger">
<b>Assumption:</b> if the value of ultimate_parent_legal_entity_name is 'do not use' (0.17%) it means not to use the entity name and not the entire row</b>
</div>

In [132]:
# 'do not use'
#sum(df["ultimate_parent_legal_entity_name"] == 'do not use') / len(df)
df.loc[df["ultimate_parent_legal_entity_name"] == 'do not use', "ultimate_parent_legal_entity_name"] = np.nan

In [133]:
# certain entries have exact data except of "compset"
# we want to take the union of all of them

grouping_columns = [col for col in df.columns if col != 'compset']

df.fillna('Group_Null', inplace=True)
result = df.groupby(grouping_columns).agg({'compset': lambda x: set(x)}).reset_index()
df = result
df.replace('Group_Null', np.nan, inplace=True)

# Feature Engineering

### Miscellaneous

Add some data related features

In [134]:
# split the date into year, month, day
df['Year'] = df['period_end_date'].dt.year
df['Month'] = df['period_end_date'].dt.month
df['Day'] = df['period_end_date'].dt.day
df['Weekday'] = df['period_end_date'].dt.dayofweek

Add variables related to ownership structure

In [135]:
df["ultimate_parent_vs_legal_entity"] = df["ultimate_parent_legal_entity_name"] != df["legal_entity_name"]
df["ultimate_parent_vs_business_entity"] = df["ultimate_parent_legal_entity_name"] != df["business_entity_doing_business_as_name"]
df["legal_entity_vs_business_entity"] = df["legal_entity_name"] != df["business_entity_doing_business_as_name"]
df["same_ownership"] = (df["legal_entity_name"] == df["ultimate_parent_legal_entity_name"]) & (df["legal_entity_name"] == df["business_entity_doing_business_as_name"])
# get the difference between the current date and the previous date
#df = df.sort_values(by=['business_entity_doing_business_as_name', 'period_end_date'])
#df['date_diff_prev'] = df['period_end_date'].diff().dt.days
#df['date_diff_prev'] = df['date_diff_prev'].fillna(7)

### Conversion Rate Modelling

<div class="alert alert-block alert-danger">
<b>Assumption:</b> likes and comments are only for the videos and pictures uploaded within the same week</b>
</div>

In [136]:
# total involevement
df["total_involvement"] = df["comments"] + df["likes"]
df["total_company_activity"] = df["pictures"] + df["videos"]

df["conversion_rate_total"] = df["total_involvement"] / df["followers"]
df["return_on_activity"] = df["total_company_activity"] / df["total_involvement"] 

# COntent type
df["ratio_of_videos"] = df["videos"] / (df["pictures"] + df["videos"])
df["ratio_of_pictures"] = df["pictures"] / (df["pictures"] + df["videos"])

# ASSUMTION: we only like videos / photos from this week
df["likes_per_picture"] = df["likes"] / df["pictures"]
df["likes_per_video"] =   df["likes"] / df["videos"]
df["comments_per_picture"] =  df["comments"] / df["pictures"] 
df["comments_per_video"] =   df["comments"] / df["videos"]

# take care of zeros
df.loc[df["pictures"] + df["videos"] == 0, "ratio_of_videos"] = np.nan
df.loc[df["pictures"] + df["videos"] == 0, "ratio_of_pictures"] = np.nan

df.loc[df["pictures"] == 0, "likes_per_picture"] = np.nan
df.loc[df["videos"] == 0, "likes_per_video"] = np.nan
df.loc[df["pictures"] == 0, "comments_per_picture"] = np.nan
df.loc[df["videos"] == 0, "comments_per_video"] = np.nan

### Competition Analysis

In [137]:
vars_to_aggreg = ["followers", "videos", "pictures", "likes", "comments"]

vars_agg_ind = ["period_end_date", "compset_group", "business_entity_doing_business_as_name"]
vars_agg_ind_small = ["period_end_date", "compset_group"]

vars_agg_ind_country = ["period_end_date", "compset_group", "domicile_country_name", "business_entity_doing_business_as_name"]
vars_agg_ind_country_small = ["period_end_date", "compset_group", "domicile_country_name"]

In [138]:
df_group_ind = df.drop_duplicates(vars_agg_ind).groupby(vars_agg_ind_small)
df_group_ind_country = df.drop_duplicates(vars_agg_ind_country).groupby(vars_agg_ind_country_small)

df_cnt_industry = pd.DataFrame(df_group_ind.count()["business_entity_doing_business_as_name"])
df_cnt_industry_country = pd.DataFrame(df_group_ind_country.count()["business_entity_doing_business_as_name"])
df_sum_industry = df_group_ind.sum()[vars_to_aggreg]
df_sum_industry_country = df_group_ind_country.sum()[vars_to_aggreg]

# rename variables before the join
df_cnt_industry.rename(lambda x: x + "_cnt_industry", axis='columns', inplace=True)
df_cnt_industry_country.rename(lambda x: x + "_cnt_industry_country", axis='columns', inplace=True)
df_sum_industry.rename(lambda x: x + "_sum_industry", axis='columns', inplace=True)
df_sum_industry_country.rename(lambda x: x + "_sum_industry_country", axis='columns', inplace=True)

In [139]:
# the join
df = df.join(df_cnt_industry, on=vars_agg_ind_small)
df = df.join(df_cnt_industry_country, on=vars_agg_ind_country_small)
df = df.join(df_sum_industry, on=vars_agg_ind_small)
df = df.join(df_sum_industry_country, on=vars_agg_ind_country_small)

In [140]:
# creating ratios
vars_to_create = ["followers", "videos", "pictures", "likes", "comments"]

for varname in vars_to_create:
    # define varnames
    # input
    aggreg_ind = varname + "_sum_industry"
    aggreg_ind_country = varname + "_sum_industry_country"
    # output
    out_agg_ind = "fraction_" + aggreg_ind
    out_agg_ind_country = "fraction_" + aggreg_ind_country
    # compute fraction
    df[out_agg_ind] = df[varname] / df[aggreg_ind]
    df[out_agg_ind_country] =  df[varname] / df[aggreg_ind_country]
    # add missing values
    df.loc[df[aggreg_ind] == 0, out_agg_ind] = np.nan
    df.loc[df[aggreg_ind_country] == 0, out_agg_ind_country] = np.nan
    # print status
    print(f"Created {out_agg_ind} from {varname} and {aggreg_ind}")
    print(f"Created {out_agg_ind_country} from {varname} and {aggreg_ind_country}")

Created fraction_followers_sum_industry from followers and followers_sum_industry
Created fraction_followers_sum_industry_country from followers and followers_sum_industry_country
Created fraction_videos_sum_industry from videos and videos_sum_industry
Created fraction_videos_sum_industry_country from videos and videos_sum_industry_country
Created fraction_pictures_sum_industry from pictures and pictures_sum_industry
Created fraction_pictures_sum_industry_country from pictures and pictures_sum_industry_country
Created fraction_likes_sum_industry from likes and likes_sum_industry
Created fraction_likes_sum_industry_country from likes and likes_sum_industry_country
Created fraction_comments_sum_industry from comments and comments_sum_industry
Created fraction_comments_sum_industry_country from comments and comments_sum_industry_country


### Lagged Variables

<div class="alert alert-block alert-danger">
<b>Assumption: </b> the first available observation for the company coincides with the first week the company has opened its instagram account
</div>

In [141]:
df.sort_values(["business_entity_doing_business_as_name", 'period_end_date'], inplace=True) 

lag_size = 1

for lag_var in ['followers', 'pictures', 'videos', 'comments']:
    print(f'Creating lag_{lag_size}_{lag_var}')
    df[f'lag_{lag_size}_{lag_var}'] = df[lag_var].shift(lag_size)
    df[f'lag_{lag_size}_date'] = df['period_end_date'].shift(lag_size)  # to check the time difference
    df[f'lag_{lag_size}_company'] = df["business_entity_doing_business_as_name"].shift(lag_size)
    df[f'timediff_{lag_size}'] = df['period_end_date'] - df[f'lag_{lag_size}_date']
    df[f'timediff_{lag_size}'] = df[f'timediff_{lag_size}'].apply(lambda x: x.days // 7)
    # set to 0 the ones where company has changed
    # ASSUMPTION HERE
    df.loc[ df[f'lag_{lag_size}_company'] != df["business_entity_doing_business_as_name"], f'lag_{lag_size}_{lag_var}']  = 0
    df.loc[ df[f'lag_{lag_size}_company'] != df["business_entity_doing_business_as_name"], f'lag_{lag_size}_date']  = np.nan
    # Change compared to the last week
    df[f'diff_{lag_size}_{lag_var}'] = df[lag_var] - df[f'lag_{lag_size}_{lag_var}']
    #df.loc[ df[f'timediff_{lag_size}'] != lag_size, f'lag_{lag_size}_{lag_var}']  = np.nan

Creating lag_1_followers
Creating lag_1_pictures
Creating lag_1_videos
Creating lag_1_comments


# Dataset enhancement

In [142]:
# ADD GINI
reload(pg)

df_gini = pg.process_gini(df)
df = pd.merge(df, df_gini, left_on=['domicile_country_name', 'Year'], right_on=['Country Name', 'Year'], how="left")

In [ ]:
df[['period_end_date', f'diff_{lag_size}_{lag_var}']]

In [121]:
output_filepath = '../data/data_preparation_output.csv'
df.to_csv(output_filepath)